<a href="https://colab.research.google.com/github/parthpanchal2402/Coursera_Capstone/blob/master/SC_toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Segmenting and Clustering Neighborhoods in Toronto###


This notebook is for the Week 3's assignment Segmenting and Clustering Neighborhoods in Toronto.

###Question 1###

In [0]:
import requests
import folium

import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

In [2]:
#Reading data
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values = 'Not assigned', flavor = 'html5lib')
print(type(df[0]))
df = df[0]

<class 'pandas.core.frame.DataFrame'>


In [0]:
#Checking columns
df.columns
df.rename({0: 'Postcode', 1: 'Borough', 2: 'Neighbourhood'}, axis = 'columns', inplace = True)

##Pre-processing data##

In [4]:
#Dealing with NaN's
print(df.isnull().sum())
df.drop(df.index[0], inplace = True)
df.dropna(axis = 0, inplace = True)
print(df.isnull().sum())
df.head()

Postal code      0
Borough         77
Neighborhood    77
dtype: int64
Postal code     0
Borough         0
Neighborhood    0
dtype: int64


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
#Checking number of boroughs and neighbourhoods
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(

len(df['Borough'].unique()),

len(df['Neighborhood'].unique())))
#Printing shape

print(df.shape)
df.to_csv('Toronto_df.csv')

The dataframe has 10 boroughs and 98 neighbourhoods.
(103, 3)


### Question 2###

In [0]:
df = pd.read_csv('Toronto_df.csv')


In [7]:
#Getting postal codes
df['Postal code'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [8]:
#Sorting the dataframe and droping its index

df = df.sort_values(by = ['Postal code'], ascending = True)
df.reset_index(inplace= True, drop = True)
df.head()

,Unnamed: 0,Postal code,Borough,Neighborhood
0,9,M1B,Scarborough,Malvern / Rouge
1,18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,27,M1E,Scarborough,Guildwood / Morningside / West Hill
3,36,M1G,Scarborough,Woburn
4,45,M1H,Scarborough,Cedarbrae


In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
tables[0].columns = tables[0].iloc[0]
df_table = tables[0]

# Dataframe will consist of 3 columns: PostalCode, Borough, and Neighborhood
df_table.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_table = df_table.iloc[1:]

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_table = df_table[df_table.Borough != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, set neighborhood name to its borough.
df_table.Neighborhood.replace('Not assigned',df_table.Borough,inplace=True)

# Combine neighborhoods with same postal code in a single row and separate with commas (as_index=False to retain columns)
df_table = df_table.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))

df_geospace = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospace.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_final = pd.merge(df_table, df_geospace, on = 'PostalCode')
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


###Question 3###